# Data Cleansing (Alumni Dataset)

#### By : Vinayak Kumar Pathak

### Importing Libraries & Data :

In [18]:
import pandas as pd
import numpy as np
import re

alumni_data = pd.read_excel(r'F:\\CleansingAssessment\\Alumni_Dataset.xlsx', sheet_name = 'Data')
sports_club = pd.read_excel(r'F:\\CleansingAssessment\\Alumni_Dataset.xlsx', sheet_name = 'Sports Club')
course_structure = pd.read_excel(r'F:\\CleansingAssessment\\Alumni_Dataset.xlsx', sheet_name = 'Course Structure')

### Previewing & performing basic formatting on Main Data :

In [ ]:
alumni_data.head()

In [ ]:
alumni_data.info()

In [19]:
"""
These columns are deleted for following reasons :
Name Blank - It is supposed to indicate whether Name of an Alumni is blank or not but it has values for only one fourth of total data, hence it is not useful.
Unnamed: 18 - It is an empty column with no values.
Unnamed: 19 - It contains either 1 or 0 with no explanation.
1 - It contains consecutive integers which is of no use.
sadye.reynolds@beer.net : This column contains email addresses which doesn't even match with the corresponding value in 'Email Address' column.

Apart from above mentioned reasons, no instructions were provided for these columns. Thus I assumed them to be insignificant.
"""

alumni_data = alumni_data.drop(['Name Blank', 'Unnamed: 18', 'Unnamed: 19', 1, 'sadye.reynolds@beer.net'], axis=1)

In [20]:
# Converting data types to best possible match
alumni_data = alumni_data.convert_dtypes()

### Dropping Rows which are not following the required rules :

##### Rule : The Name fields ('First Name' and 'Last Name') should not be blank

In [ ]:
# Creating a subset dataframe from alumni_data with rows having Null values for 'First Name' or 'Last Name' column.
empty_names = alumni_data[(alumni_data['First Name'].isnull()) | (alumni_data['Last Name'].isnull())]

# Checking if the dataframe has atleast one row
if (empty_names.shape[0] > 0) :

    # Adding a column in the subset dataframe
    empty_names.loc[:,'Reason'] = "Either First Name or Last Name is Blank"

##### Rule : All Email address should be unique, which means no more than one record should have same email address

In [ ]:
# Creating a subset dataframe from alumni_data with rows having duplicate values for 'Email Address' column.
dup_email = alumni_data.loc[alumni_data['Email Address'].duplicated(), :]

if (dup_email.shape[0] > 0) :
    dup_email.loc[:,'Reason'] = "Contains duplicate Email"

##### Rule : All Email address should be valid which means it should be of the format abc@xyz.pqr

In [23]:
# Regular Expression for valid format of 'Email Address'
pattern = re.compile('[a-zA-Z0-9._-]+@[a-zA-Z]+.[a-zA-Z]+')

# Creating a subset dataframe from alumni_data with rows having email address of valid format.
right_email = alumni_data[alumni_data['Email Address'].str.match(pattern, na=False)]

# Creating a subset dataframe from alumni_data with rows not having email address of valid format.
wrong_email = alumni_data.drop(right_email.index)

if (wrong_email.shape[0] > 0) :
    wrong_email.loc[:,'Reason'] = "Wrong Format of Email Address"

# Setting "right_email" to None as it is no longer required
right_email = None

##### Rule : 'College Name' should not be blank if 'Degree' and 'Field of Study' is given

In [24]:
# Creating a subset dataframe from alumni_data where 'College Name' is blank when 'Degree' and 'Field of Study' is given
blank_college_name = alumni_data[(alumni_data['College Name'].isnull()) & (alumni_data['Degree'].notnull()) & (alumni_data['Field of Study'].notnull())]

if (blank_college_name.shape[0] > 0) :
    blank_college_name.loc[:,'Reason'] = "College Name is Empty even when Degree and Field of Study is given"

##### Rule : The sheet "Course Structure" is the table with the course details. The data in the columns 'College Name', 'Degree' & 'Field of Study' together should match with the one in the sheet "Course Structure"

In [25]:
"""
Merging (Left Join) "alumni_data" with "course_structure" on columns 'College Name', 'Degree' and 'Field of Study' and adding an indicator column 'Exists'.
Indicator column tells if the row (subset on which both dataframes are merged) in "alumni_data" is present in "course_structure".
If the value is 'both' then it exists, otherwise it does not.
reset_index() method on "alumni_data" and set_index('index') method on the merged dataframe is used to make sure that the original index of the rows does not change .
Since we performed left join, "cn_d_fos" contains all values from "alumni_data".
"""
cn_d_fos = pd.merge(alumni_data.reset_index(), course_structure, on=['College Name', 'Degree', 'Field of Study'], how='left', indicator='Exists').set_index('index')

# Converting the values of 'Exists' column to boolean.
cn_d_fos['Exists'] = np.where(cn_d_fos.Exists == 'both', True, False)

# Creating a subset dataframe from "cn_d_fos" with rows having False value for 'Exists' column
invalid_cn_d_fos = cn_d_fos[cn_d_fos['Exists'] == False]

# Dropping the 'Exists' column since it is no longer required.
invalid_cn_d_fos = invalid_cn_d_fos.drop(['Exists'], axis = 1)

if (invalid_cn_d_fos.shape[0] > 0) :
    invalid_cn_d_fos.loc[:,'Reason'] = "College Name, Degree & Field of Study together does not match with the Course Structure sheet"

# Setting "cn_d_fos" to None as it is no longer required
cn_d_fos = None

##### Rule : 'Employer Name' cannot be blank if the 'Job Domain' or 'Job Title' are given

In [ ]:
# Creating a subset dataframe from alumni_data where 'Employer Name' is blank when 'Job Domain' or 'Job Title' is given
blank_employer_name = alumni_data[(alumni_data['Employer Name'].isnull()) & ((alumni_data['Job Domain'].notnull()) | (alumni_data['Job Title'].notnull()))]

if (blank_employer_name.shape[0] > 0) :
    blank_employer_name.loc[:,'Reason'] = "Employer Name is Empty even when Job Domain or Job Title is given"

##### Rule : 'Date of Birth' should be of the format mm/dd/yyyy or mm-dd-yyyy and all three values must be given i.e., Day, Month and Year

In [27]:
# Regular Expression for valid format of 'Date of Birth'
pattern2 = re.compile('[01]{0,1}[0-9](/|-)[0123]{0,1}[0-9](/|-)(19|20)[0-9]{2}')

# Creating a subset dataframe from alumni_data with rows having date of birth of valid format.
right_dob = alumni_data[alumni_data['Date of birth'].str.match(pattern2, na=False)]

# Creating a subset dataframe from alumni_data with rows not having date of birth of valid format.
wrong_dob = alumni_data.drop(right_dob.index)

if (wrong_dob.shape[0] > 0) :
    wrong_dob.loc[:,'Reason'] = "Wrong Format of Date of Birth"

# Setting "right_dob" to None as it is no longer required
right_dob = None

##### Rule : 'Year of Graduation' and 'Year of Entry' has to be valid
Bonus Rule : Year of Birth should be atleast before 2014 (Even if we assume a case similar to the prodigious case of Michael Kearney)

In [28]:
# Regular Expression for Year
pattern3 = re.compile('([12][09][0-9]{2})')

# Changing data type of 'Date of birth' column to string.
alumni_data['Date of birth'] = alumni_data['Date of birth'].astype(str)

# Adding a 'YOB' (Year of Birth) column for ease of calculation
alumni_data.loc[:,'YOB'] = alumni_data['Date of birth'].str.findall(pattern3)

# Converting data type from list to string.
alumni_data['YOB'] = [''.join(map(str, l)) for l in alumni_data['YOB']]

# Replacing empty strings with N/A values
alumni_data['YOB'] = alumni_data['YOB'].replace('', np.nan)

# Converting data type from string to integer
alumni_data['YOB'] = alumni_data['YOB'].astype('Int64')

"""
Following assumptions were made to check the validity of 'Year of Graduation' and 'Year of Entry' columns -
1. 'Year of Entry' must be greater than Year of Birth
2. 'Year of Graduation' must be greater than 'Year of Entry'
3. 'Year of Graduation' must be lesser than the current year.
"""
# Creating a subset dataframe from alumni_data with rows not having valid values for 'Year of Graduation' and 'Year of Entry' columns
invalid_year = alumni_data[(alumni_data['Year of Entry'] < alumni_data['YOB']) & (alumni_data['Year of Graduation'] < alumni_data['Year of Entry']) & (alumni_data['Year of Graduation'] > 2023)]

# Creating a subset dataframe from alumni_data with rows having unrealistic values for Year of Birth
invalid_yob = alumni_data[alumni_data['YOB'] > 2014]

# Dropping the 'YOB' column since it is no longer required.
alumni_data = alumni_data.drop(['YOB'], axis=1)
invalid_year = invalid_year.drop(['YOB'], axis=1)
invalid_yob = invalid_yob.drop(['YOB'], axis=1)

if (invalid_year.shape[0] > 0) :
    invalid_year.loc[:,'Reason'] = "Year of Graduation or Year of Entry is not valid"

if (invalid_yob.shape[0] > 0) :
    invalid_yob.loc[:,'Reason'] = "Year of birth in Date of birth column has an unrealistic value"

##### Rule : Sheet "Sports Club" has a list of clubs in the school. The data in the sheet "Assignment 1 - Data" should have only the clubs listed in the sheet "Sports Club"

In [29]:
# Creating a subset dataframe from alumni_data where values of 'Club1 ', 'CLub2', 'Club3' and 'Club4' columns matches with "sports_club" sheet
valid_sports_club = alumni_data[(alumni_data['Club1 '].isin(sports_club['Sports Club'])) & (alumni_data['CLub2'].isin(sports_club['Sports Club'])) & (alumni_data['Club3'].isin(sports_club['Sports Club'])) & (alumni_data['Club4'].isin(sports_club['Sports Club']))]

# Creating a subset dataframe from alumni_data where values of 'Club1 ', 'CLub2', 'Club3' and 'Club4' columns doesn't matches with "sports_club" sheet
invalid_sports_club = alumni_data.drop(valid_sports_club.index)

if (invalid_sports_club.shape[0] > 0) :
    invalid_sports_club.loc[:,'Reason'] = "Name of the Club is not listed in Sports Club sheet"

# Setting "valid_sports_club" to None as it is no longer required
valid_sports_club = None

##### Rule : Cells with value N/A are not allowed

In [30]:
# Creating a subset dataframe from alumni_data with rows not having N/A values
no_na_val = alumni_data.dropna()

# Creating a subset dataframe from alumni_data with rows having N/A values
na_val = alumni_data.drop(no_na_val.index)

if (na_val.shape[0] > 0) :
    na_val.loc[:,'Reason'] = "Cell(s) contains N/A value"


### Concatenating dropped rows in a single DataFrame

In [34]:
# reset_index() method is used to preserve the original index of rows for easy reference in original file.
dropped_rows = pd.concat([empty_names, dup_email, wrong_email, blank_college_name, invalid_cn_d_fos, blank_employer_name, wrong_dob, invalid_year, invalid_yob, invalid_sports_club, na_val], axis = 0).reset_index()
dropped_rows = dropped_rows.sort_values(by=['index'])

### Saving DataFrame into CSV

In [35]:
dropped_rows.to_csv(r'F:\\CleansingAssessment\\DroppedRows_AlumniDataset.csv', index=False)